In [1]:
import os
import sys
import warnings

PROJ_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.append(PROJ_PATH) 

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import itertools
from pandas.api.types import CategoricalDtype

from datetime import date 
from datetime import datetime, timedelta 
import datetime as DT 
import time

import xgboost as xgb 
from boruta import BorutaPy

import helper_functions as hfs 
from scipy.stats import linregress 
import math
from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.metrics import confusion_matrix


from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K


import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_fscore_support
import seaborn as sns


import keras
import tensorflow


warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)  # to see all columns of dataframe
pd.set_option("display.max_rows", None)  # to see all rows of dataframe
pd.set_option("display.max_colwidth", None)  # to see the full (un-truncated) dataframe field/cell value 

In [2]:
import configparser

env_config = configparser.ConfigParser()
env_config.read(f'{PROJ_PATH}/kye/env_config.ini')

# <<<<<<<<<<<<<<< >>>>>>>>>>>>>>>>>>>>>>> #

SPLIT_DATE = pd.to_datetime(env_config['ENV_VARS']['SPLIT_DATE'])
ML_DATA_FOLDER_PATH = env_config['LOCAL_PATHS']['ML_DATA_FOLDER_PATH']
PROCESSING_TEMP_FOLDER_PATH = env_config['LOCAL_PATHS']['PROCESSING_TEMP_FOLDER_PATH'] 

OUTPUTS_FOLDER_PATH = env_config['LOCAL_PATHS']['OUTPUT_FOLDER_PATH'] + 'D1/'

#@@@@@@ **********************<<<<<<<<<<<<#

print("SPLIT_DATE: ", SPLIT_DATE)

TODAY = str(date.today())
FROM_DATE = pd.to_datetime(date.today(), format='%Y-%m-%d') - DT.timedelta(days=((365*4) + 60)) # "2016-01-01"
TO_DATE = str(date.today())
TWO_YEARS_BEFORE_FROM_DATE = pd.to_datetime(FROM_DATE, format='%Y-%m-%d') - DT.timedelta(days=730)
FROM_DATE

SPLIT_DATE:  2023-08-09 00:00:00


Timestamp('2019-06-11 00:00:00')

In [4]:
files_in_ML_DATA = os.listdir(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}")  
files_in_ML_DATA

['all_stocks_ohlcv.pkl',
 'stocks_current_fundamentals_1340_2023-08-09.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints']

In [5]:
files_in_PROCESSING_TEMP = os.listdir(f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}") 
files_in_PROCESSING_TEMP 

['RIPOs_df.pkl',
 'analytics_feature_ind.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints',
 'OHLCV_all_stocks_ohlcv_data_prep.pkl',
 'predicted_EPs.pkl']

In [6]:
files_in_OUTPUTS = os.listdir(f"{PROJ_PATH}/{OUTPUTS_FOLDER_PATH}") 
files_in_OUTPUTS 

['.ipynb_checkpoints']

In [7]:
final = pd.read_pickle( f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/{[x for x in files_in_PROCESSING_TEMP if 'OHLCV_all_stocks_ohlcv_data_prep' in x][0]}") 
final = final[final['Ticker'].apply(lambda x: '-' not in x)].reset_index(drop=True)
if 'Unnamed: 0' in final.columns:
    del final['Unnamed: 0']
print("#Tickers: ", final['Ticker'].nunique())
print(final.shape)
final.tail(2) 

#Tickers:  1340
(366970, 331)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [9]:
duplicate_indices = [x for x in final[['Ticker','Date']][final[['Ticker','Date']].duplicated()].index]
final_train_test_dummies = final.drop(final.index[duplicate_indices]).reset_index(drop=True)
print(final_train_test_dummies.shape)
final_train_test_dummies.tail(2) 

(366970, 331)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [10]:
categorical_data = final.select_dtypes(include=['category']) 
print(categorical_data.shape) 
categorical_data.head(2) 


(366970, 0)


""
0
1


In [11]:
missing_df = hfs.missing_values_info(final_train_test_dummies) 
print(missing_df.shape) 
missing_df 


(0, 3)


,Feature,#missing_values,%missing_values


In [12]:
# converting 0/1 columns to boolean type
bool_cols_01_vals = []
for col in final_train_test_dummies.columns:
    #print(col)
    unique_vals = final_train_test_dummies[col].unique()
    if (len(unique_vals) == 2):
        if sorted(unique_vals) == [0,1]:
            bool_cols_01_vals.append(col)
            final_train_test_dummies[col] = final_train_test_dummies[col].astype(bool)

print("Bool cols: ", len(bool_cols_01_vals))
final_train_test_dummies[bool_cols_01_vals].tail(2) 

Bool cols:  36


,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,straight_down,straight_up,close_high_avg_inline_GSPC,Exchange_NASDAQ,Exchange_NYSE,Exchange_NYSE MKT,General__AddressData.Country_Cayman Islands,General__AddressData.Country_China,General__AddressData.Country_Hong Kong,General__AddressData.Country_Israel,General__AddressData.Country_Other,General__AddressData.Country_Taiwan,General__AddressData.Country_United States,General__Sector_,General__Sector_Basic Materials,General__Sector_Communication Services,General__Sector_Consumer Cyclical,General__Sector_Consumer Defensive,General__Sector_Energy,General__Sector_Financial Services,General__Sector_Healthcare,General__Sector_Industrials,General__Sector_Real Estate,General__Sector_Technology,General__Sector_Utilities,price_cat_GT_20,price_cat_GT_2_LT_5,price_cat_GT_5_LT_20,price_cat_LT_2
366968,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
366969,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False


In [13]:
no_change_cols = []
for col in final_train_test_dummies.columns:
    if final_train_test_dummies[col].nunique() == 1:
        no_change_cols.append(col)

print("These columns are useless so removing: ", no_change_cols)
print(final_train_test_dummies.shape)
final_train_test_dummies.drop(columns=no_change_cols, inplace=True)
print(final_train_test_dummies.shape) 


These columns are useless so removing:  []
(366970, 331)
(366970, 331)


In [14]:
final = final_train_test_dummies.copy()


In [15]:
def create_lag_feature_for_DFuture(df, groupbyCol, col_name, days_in_future = 1):
    for lagN in range(1, days_in_future):
        df[f'{col_name}_DFuture{lagN}'] = df.groupby(groupbyCol)[col_name].shift(lagN) 
    
    return df.groupby(['Ticker']).apply(lambda x: x.iloc[days_in_future-1:]) 

In [16]:
def convert_floats_to_float32(df):
    col_type = df.dtypes.reset_index()
    col_type.columns = ['Column_Name', 'type']
    float_columns = list(col_type["Column_Name"][(col_type['type']=="float64")
                                                ].values)
    for col in float_columns:
        #df[col] = np.round(df[col].astype(np.float32), N_decimals)
        df[col] = df[col].astype(np.float32)
        
    return df.round(decimals=3)

print(final.info())
final = convert_floats_to_float32(final)
final.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366970 entries, 0 to 366969
Columns: 331 entries, company_Name to price_cat_LT_2
dtypes: bool(36), datetime64[ns](1), float32(285), float64(1), int64(6), object(2)
memory usage: 439.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366970 entries, 0 to 366969
Columns: 331 entries, company_Name to price_cat_LT_2
dtypes: bool(36), datetime64[ns](1), float32(286), int64(6), object(2)
memory usage: 438.2+ MB


In [17]:
hfs.missing_values_info(final) 


,Feature,#missing_values,%missing_values


In [18]:
final['next_8_days_close_list'] = list(zip(final.groupby("Ticker")['Adj_Close'].shift(-1), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-2), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-3), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-4),
                                          final.groupby("Ticker")['Adj_Close'].shift(-5), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-6), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-7), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-8)
                                         ))

final['next_8_days_close_list'] = final['next_8_days_close_list'].apply(lambda x: list(x))
final['next_8_days_close_list'] = final['next_8_days_close_list'].apply(
                                    lambda lst: [0 if np.isnan(val) else val for val in lst])

final['next_5_days_close_list'] = final['next_8_days_close_list'].apply(lambda lst: lst[0:5] )

final['Adj_Close_plus_Xper'] = final['Adj_Close'] * 1.3 

final['days_close_surpasses_Longer'] = final.apply(lambda row: len( list ( filter ( lambda i: i > row['Adj_Close_plus_Xper'], row['next_8_days_close_list'] ) ) ) , axis=1) 
final['days_close_surpasses_Shorter'] = final.apply(lambda row: len( list ( filter ( lambda i: i > row['Adj_Close_plus_Xper'], row['next_5_days_close_list'] ) ) ) , axis=1) 

final['next_8_days_High_list'] = list(zip(final.groupby("Ticker")['Adj_High'].shift(-1), 
                                          final.groupby("Ticker")['Adj_High'].shift(-2), 
                                          final.groupby("Ticker")['Adj_High'].shift(-3), 
                                          final.groupby("Ticker")['Adj_High'].shift(-4),
                                          final.groupby("Ticker")['Adj_High'].shift(-5), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-6), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-7), 
                                          final.groupby("Ticker")['Adj_Close'].shift(-8)
                                         ))

final['next_8_days_High_max'] = final['next_8_days_High_list'].apply(lambda x: max(x))

final.drop( columns = ['next_8_days_close_list', 'next_5_days_close_list' ,'Adj_Close_plus_Xper', 'next_8_days_High_list' ], inplace=True) 

In [19]:
final.tail() 

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [20]:
special_test_set = final[final['Date'].isin(sorted(final['Date'].unique())[-15:])
                                               ].reset_index(drop=True)

In [21]:
sorted(special_test_set['Date'].unique())

[numpy.datetime64('2023-07-20T00:00:00.000000000'),
 numpy.datetime64('2023-07-21T00:00:00.000000000'),
 numpy.datetime64('2023-07-24T00:00:00.000000000'),
 numpy.datetime64('2023-07-25T00:00:00.000000000'),
 numpy.datetime64('2023-07-26T00:00:00.000000000'),
 numpy.datetime64('2023-07-27T00:00:00.000000000'),
 numpy.datetime64('2023-07-28T00:00:00.000000000'),
 numpy.datetime64('2023-07-31T00:00:00.000000000'),
 numpy.datetime64('2023-08-01T00:00:00.000000000'),
 numpy.datetime64('2023-08-02T00:00:00.000000000'),
 numpy.datetime64('2023-08-03T00:00:00.000000000'),
 numpy.datetime64('2023-08-04T00:00:00.000000000'),
 numpy.datetime64('2023-08-07T00:00:00.000000000'),
 numpy.datetime64('2023-08-08T00:00:00.000000000'),
 numpy.datetime64('2023-08-09T00:00:00.000000000')]

In [22]:
final = final.dropna().reset_index(drop=True)
sorted(final['Date'].unique())[-15:] 

[numpy.datetime64('2023-07-19T00:00:00.000000000'),
 numpy.datetime64('2023-07-20T00:00:00.000000000'),
 numpy.datetime64('2023-07-21T00:00:00.000000000'),
 numpy.datetime64('2023-07-24T00:00:00.000000000'),
 numpy.datetime64('2023-07-25T00:00:00.000000000'),
 numpy.datetime64('2023-07-26T00:00:00.000000000'),
 numpy.datetime64('2023-07-27T00:00:00.000000000'),
 numpy.datetime64('2023-07-28T00:00:00.000000000'),
 numpy.datetime64('2023-07-31T00:00:00.000000000'),
 numpy.datetime64('2023-08-01T00:00:00.000000000'),
 numpy.datetime64('2023-08-02T00:00:00.000000000'),
 numpy.datetime64('2023-08-03T00:00:00.000000000'),
 numpy.datetime64('2023-08-04T00:00:00.000000000'),
 numpy.datetime64('2023-08-07T00:00:00.000000000'),
 numpy.datetime64('2023-08-08T00:00:00.000000000')]

In [23]:
missing_df = hfs.missing_values_info(final) 
print(missing_df.shape) 
missing_df 


(0, 3)


,Feature,#missing_values,%missing_values


# Classification

In [24]:
final_train_test_high_cls = final.copy() # final_train_test_dummies.copy() 
print(final_train_test_high_cls.shape) 

final_train_test_high_cls['long_swingable'] = np.where( ( (final_train_test_high_cls['days_close_surpasses_Longer'] >= 4) |
                                                      (final_train_test_high_cls['days_close_surpasses_Shorter'] >= 2) 
                                                    ),
                                                   1,
                                                   0)

print(final_train_test_high_cls.shape) 
final_train_test_high_cls.tail(2) 

(365630, 334)
(365630, 335)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [25]:
final_train_test_high_cls['Adj_Close'].describe()


count    365630.000000
mean         16.612829
std          27.958410
min           0.010000
25%           1.920000
50%           6.740000
75%          21.209999
max        1680.000000
Name: Adj_Close, dtype: float64

In [26]:
final_train_test_high_cls['long_swingable'].value_counts() 


0    359568
1      6062
Name: long_swingable, dtype: int64

In [27]:
hfs.missing_values_info(final_train_test_high_cls) 


,Feature,#missing_values,%missing_values


In [28]:
final_train_test_high_cls['Date'].nunique()


281

In [29]:
final_train_test_high_cls.tail(2)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [30]:
#SPLIT_DATE = "2020-10-15" # (SPLIT_DATE + 1)  would be the day you want to predict for
print("SPLIT_DATE: ",SPLIT_DATE )

result_df_cls = final_train_test_high_cls[final_train_test_high_cls['Date'].isin(sorted(final['Date'].unique())[-13:-8])
                                         ].reset_index(drop=True)
result_df_cls = result_df_cls[['Ticker','Date','long_swingable']]
print("result_df shape: ", result_df_cls.shape)

final_train_test_high_cls.drop(columns=['days_close_surpasses_Longer', 'days_close_surpasses_Shorter', 'next_8_days_High_max'], inplace=True)

#print("train_test shape: ", final_train_test_high_cls.shape)
#final_train_test_high_cls = final_train_test_high_cls[req_features]
print("train_test shape: ", final_train_test_high_cls.shape)

final_train_high_cls = final_train_test_high_cls[final_train_test_high_cls['Date'].isin(sorted(final['Date'].unique())[:-21])].reset_index(drop=True)
#final_train_high_cls = final_train_high_cls.dropna().reset_index(drop=True)
print("train shape: ", final_train_high_cls.shape) 

final_test_high_cls = final_train_test_high_cls[final_train_test_high_cls['Date'].isin(sorted(final['Date'].unique())[-13:-8])
                                               ].reset_index(drop=True)
#final_test_high_cls = final_test_high_cls.dropna().reset_index(drop=True)
print("test shape: ", final_test_high_cls.shape) 


SPLIT_DATE:  2023-08-09 00:00:00
result_df shape:  (6700, 3)
train_test shape:  (365630, 332)
train shape:  (337490, 332)
test shape:  (6700, 332)


In [31]:
print(f'train upto: {final_train_high_cls["Date"].max()}')
print(f'result_df_cls dates: {result_df_cls["Date"].min()} {result_df_cls["Date"].max()}')
print(f'test dates: {final_test_high_cls["Date"].min()} {final_test_high_cls["Date"].max()}')
print(f'special_test_set dates: {special_test_set["Date"].min()} {special_test_set["Date"].max()}')


train upto: 2023-07-10 00:00:00
result_df_cls dates: 2023-07-21 00:00:00 2023-07-27 00:00:00
test dates: 2023-07-21 00:00:00 2023-07-27 00:00:00
special_test_set dates: 2023-07-20 00:00:00 2023-08-09 00:00:00


In [32]:
sorted(final['Date'].unique())[-22:]

[numpy.datetime64('2023-07-10T00:00:00.000000000'),
 numpy.datetime64('2023-07-11T00:00:00.000000000'),
 numpy.datetime64('2023-07-12T00:00:00.000000000'),
 numpy.datetime64('2023-07-13T00:00:00.000000000'),
 numpy.datetime64('2023-07-14T00:00:00.000000000'),
 numpy.datetime64('2023-07-17T00:00:00.000000000'),
 numpy.datetime64('2023-07-18T00:00:00.000000000'),
 numpy.datetime64('2023-07-19T00:00:00.000000000'),
 numpy.datetime64('2023-07-20T00:00:00.000000000'),
 numpy.datetime64('2023-07-21T00:00:00.000000000'),
 numpy.datetime64('2023-07-24T00:00:00.000000000'),
 numpy.datetime64('2023-07-25T00:00:00.000000000'),
 numpy.datetime64('2023-07-26T00:00:00.000000000'),
 numpy.datetime64('2023-07-27T00:00:00.000000000'),
 numpy.datetime64('2023-07-28T00:00:00.000000000'),
 numpy.datetime64('2023-07-31T00:00:00.000000000'),
 numpy.datetime64('2023-08-01T00:00:00.000000000'),
 numpy.datetime64('2023-08-02T00:00:00.000000000'),
 numpy.datetime64('2023-08-03T00:00:00.000000000'),
 numpy.datet

In [33]:
"""
final training dates:
train: remove last 8 days
test: last one day

validation training:
train: remove last -13 - 8 i.e. -21 days
validation testset dates: 
-13 to -8 days ( after removing last 8 days, take last 5 days)
- 21st to 27th June
"""

'\nfinal training dates:\ntrain: remove last 8 days\ntest: last one day\n\nvalidation training:\ntrain: remove last -13 - 8 i.e. -21 days\nvalidation testset dates: \n-13 to -8 days ( after removing last 8 days, take last 5 days)\n- 21st to 27th June\n'

In [34]:
result_df_cls['Date'].unique()

array(['2023-07-21T00:00:00.000000000', '2023-07-24T00:00:00.000000000',
       '2023-07-25T00:00:00.000000000', '2023-07-26T00:00:00.000000000',
       '2023-07-27T00:00:00.000000000'], dtype='datetime64[ns]')

In [35]:
final_test_high_cls['long_swingable'].value_counts() 


0    6592
1     108
Name: long_swingable, dtype: int64

In [36]:
final_train_high_cls["long_swingable"].value_counts()

0    331725
1      5765
Name: long_swingable, dtype: int64

In [37]:
class_weights={}
class_percentages = final_train_high_cls["long_swingable"].value_counts() / len(final_train_high_cls["long_swingable"])
print(class_percentages)
for i in range(len(class_percentages)):
    class_weights[i] = class_percentages[i]
class_weights

0    0.982918
1    0.017082
Name: long_swingable, dtype: float64


{0: 0.9829180123855522, 1: 0.017081987614447836}

In [38]:
class_percentages[0]

0.9829180123855522

In [39]:
def train_DNN_and_predict(df_train_data, df_test, special_test_set, class_weights_recent):

    def custom_f1_score(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (actual_positives + K.epsilon())
        f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_score


    df_train = df_train_data.copy()
    
    features_list = df_train.columns[1:-1].tolist()
    target_col = 'long_swingable'

    # Prepare the feature matrix X and target variable y
    X = df_train[features_list].values
    y = df_train[target_col].values

    # Convert the target variable y to integers using the mapping
    #y_encoded = np.array([signal_mapping[signal] for signal in y])

    # Check for missing values in the data
    #if np.isnan(X).any() or np.isnan(y_encoded).any():
    #    raise ValueError("Found missing values in the input data or target variable.")

    # Scale the features using MinMaxScaler
    scaler = MinMaxScaler()
    # X_scaled = scaler.fit_transform(X)
    scaler.fit(X)

    X_scaled = scaler.transform(X)

    # Split the data into train and test sets
    X_train = X_scaled
    y_train = y

    X_test = scaler.transform(df_test[features_list].values)
    XSpe_test = scaler.transform(special_test_set[features_list].values)
    
    #y_test = np.array([signal_mapping[signal] for signal in df_test[target_col].values])


    # Build the DNN model
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.3))
    model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.25))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=len(np.unique(y)), activation='softmax'))  # Use 'softmax' activation for multi-class classification
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy', custom_f1_score])

    # Train the DNN model
    model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, class_weight=class_weights_recent)

    ## Evaluate the model on the test set
    #loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    #print(f'Test loss: {loss}, Test accuracy: {accuracy}')

    # Make predictions on the test set
    y_pred = model.predict(X_test)
    special_test_set_pred = model.predict(XSpe_test)
    
    return y_pred, special_test_set_pred
    
pred_test_cls, special_test_set_pred = train_DNN_and_predict(final_train_high_cls.iloc[:, 3:], final_test_high_cls.iloc[:, 3:], special_test_set, class_weights)
print(len(pred_test_cls)) 
print(len(special_test_set_pred)) 

Epoch 1/20


2023-08-09 16:29:52.667352: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2637/2637 [==============================] - 3s 963us/step - loss: 0.0048 - accuracy: 0.9828 - custom_f1_score: 0.0333
Epoch 2/20
2637/2637 [==============================] - 3s 1ms/step - loss: 0.0030 - accuracy: 0.9829 - custom_f1_score: 0.0333
Epoch 3/20
2637/2637 [==============================] - 2s 944us/step - loss: 0.0028 - accuracy: 0.9829 - custom_f1_score: 0.0334
Epoch 4/20
2637/2637 [==============================] - 3s 993us/step - loss: 0.0027 - accuracy: 0.9829 - custom_f1_score: 0.0333
Epoch 5/20
2637/2637 [==============================] - 2s 930us/step - loss: 0.0026 - accuracy: 0.9829 - custom_f1_score: 0.0334
Epoch 6/20
2637/2637 [==============================] - 3s 1ms/step - loss: 0.0025 - accuracy: 0.9829 - custom_f1_score: 0.0333
Epoch 7/20
2637/2637 [==============================] - 3s 948us/step - loss: 0.0025 - accuracy: 0.9829 - custom_f1_score: 0.0333
Epoch 8/20
2637/2637 [==============================] - 3s 1ms/step - loss: 0.0025 - accuracy: 0.9829 - c

In [41]:
# train preds
#result_df_cls_feed_train = final_train_high_cls[['Ticker','Date']]
#result_df_cls_feed_train['Tomo_Close_cls_proba'] = [x[1] for x in pred_train_cls]

classes = [0,1]
    
result_df_cls_feed_test = result_df_cls.copy()

result_df_cls_feed_test[f'Tomo_Close_proba_0'] = [x[0] for x in pred_test_cls]
result_df_cls_feed_test[f'Tomo_Close_proba_1'] = [x[1] for x in pred_test_cls]

print(result_df_cls_feed_test.shape)
print(final_test_high_cls.shape)
if result_df_cls_feed_test.shape[0] != final_test_high_cls.shape[0]:
    print("##### #Rows doesn't match, fix this issue @@@@@@@@@@@@@@") 

#result_df_cls_feed.to_csv("./input_data/ML_Data/Cls_Adj_High_output_proba2.csv", index=False) 


(6700, 5)
(6700, 332)


In [42]:
result_df_cls_feed_test_copy = result_df_cls_feed_test.copy()

#result_df_cls_feed_test_copy = result_df_cls_feed_test_copy.head(int(.1 * result_df_cls_feed_test_copy.shape[0]))


In [53]:
analytics_df1 = result_df_cls_feed_test_copy.copy()
final_copy = final.copy()
final_copy['Tomo_Open'] = final_copy.groupby('Ticker')['Adj_Open'].shift(-1)

analytics_df1 = pd.merge( analytics_df1, final_copy[['Ticker','Date', 'Adj_Open', 'Adj_Low', 'Adj_High', 'Tomo_Open', 'Adj_Close', 'Adj_Vol',
                               'price_cat_GT_20', 'price_cat_GT_2_LT_5','price_cat_GT_5_LT_20', 'price_cat_LT_2', 'ADR_percent_5d', 
                                'ADR_percent_20d', 'OHLC_price', 'days_close_surpasses_Shorter', 'next_8_days_High_max','Adj_Vol_EMA_prev_5d', 
                                'Adj_Vol_EMA_prev_20d', 'EMA_9_DIV_MA50', 'EMA_5_DIV_EMA_20', 
                                'EMA_20', 'Adj_Close_MA20','Adj_Close_MA50','Adj_Close_MA100','Adj_Close_MA200']], 
                         how='left' , on=['Ticker','Date'])

def get_price_cat(price_cat_LT_2, price_cat_GT_2_LT_5, price_cat_GT_5_LT_20, price_cat_GT_20):
    if price_cat_LT_2:
        return 'LT_2'
    elif price_cat_GT_2_LT_5:
        return 'GT_2_LT_5'
    elif price_cat_GT_5_LT_20:
        return 'GT_5_LT_20'
    else:
        return 'GT_20'

analytics_df1['price_cat'] = analytics_df1.apply(lambda row: 
                                            get_price_cat( row['price_cat_LT_2'], 
                                                           row['price_cat_GT_2_LT_5'], 
                                                           row['price_cat_GT_5_LT_20'], 
                                                           row['price_cat_GT_20']), axis=1)

analytics_df1['stop_point'] = analytics_df1['Adj_Open'] - ( 0.7 * analytics_df1['Adj_Open'] * (analytics_df1['ADR_percent_5d'] / 100) )

analytics_df1['stopped_out'] = analytics_df1['Adj_Low'] > analytics_df1['stop_point']

analytics_df1['Adj_Open_before_1d'] =  analytics_df1.groupby("Ticker")['Adj_Open'].shift(1) 
analytics_df1['Adj_High_before_1d'] =  analytics_df1.groupby("Ticker")['Adj_High'].shift(1) 
analytics_df1['Adj_Close_before_1d'] =  analytics_df1.groupby("Ticker")['Adj_Close'].shift(1) 


analytics_df1 = analytics_df1[ (analytics_df1['ADR_percent_5d'] > 5.5) &
                               (analytics_df1['ADR_percent_20d'] > 8) &
                               ( ( analytics_df1['Adj_Close'] >= analytics_df1['Adj_Close_before_1d'] ) |
                                 ( analytics_df1['Adj_Open'] >= analytics_df1['Adj_Close_before_1d'] ) 
                               ) &
                              (analytics_df1['Adj_Close']>= 1) &
                              (analytics_df1['Adj_Close']<= 250) 
    
                            ].reset_index(drop=True) 

print(analytics_df1.shape)
analytics_df1 = analytics_df1.sort_values(by=['Date','price_cat','Tomo_Close_proba_1'],ascending=[True,True,False]).reset_index(drop=True)
analytics_df1 = analytics_df1.groupby(['Date','price_cat']).head(10).reset_index(drop=True)

analytics_df1['long_swingable'] = np.where( (analytics_df1['next_8_days_High_max'] >= 1.3 * analytics_df1['Adj_Close']),
                                              True,
                                              False)

print(analytics_df1['long_swingable'].value_counts())

analytics_df1.shape 


(487, 35)
False    114
True      24
Name: long_swingable, dtype: int64


(138, 35)

In [54]:
analytics_df1['Date'].unique()

array(['2023-07-24T00:00:00.000000000', '2023-07-25T00:00:00.000000000',
       '2023-07-26T00:00:00.000000000', '2023-07-27T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [55]:
special_test_set['Date'].unique()

array(['2023-07-20T00:00:00.000000000', '2023-07-21T00:00:00.000000000',
       '2023-07-24T00:00:00.000000000', '2023-07-25T00:00:00.000000000',
       '2023-07-26T00:00:00.000000000', '2023-07-27T00:00:00.000000000',
       '2023-07-28T00:00:00.000000000', '2023-07-31T00:00:00.000000000',
       '2023-08-01T00:00:00.000000000', '2023-08-02T00:00:00.000000000',
       '2023-08-03T00:00:00.000000000', '2023-08-04T00:00:00.000000000',
       '2023-08-07T00:00:00.000000000', '2023-08-08T00:00:00.000000000',
       '2023-08-09T00:00:00.000000000'], dtype='datetime64[ns]')

In [56]:
special_test_set['Date'].unique()[-9:]

array(['2023-07-28T00:00:00.000000000', '2023-07-31T00:00:00.000000000',
       '2023-08-01T00:00:00.000000000', '2023-08-02T00:00:00.000000000',
       '2023-08-03T00:00:00.000000000', '2023-08-04T00:00:00.000000000',
       '2023-08-07T00:00:00.000000000', '2023-08-08T00:00:00.000000000',
       '2023-08-09T00:00:00.000000000'], dtype='datetime64[ns]')

In [57]:
Weeks_in_analytics_df = analytics_df1['Date'].nunique() / 5 
Weeks_in_analytics_df

0.8

In [58]:
analytics_df1['long_swingable'].value_counts()

False    114
True      24
Name: long_swingable, dtype: int64

In [65]:
sorted(analytics_df1['Tomo_Close_proba_1'], reverse=True)[68]
analytics_df1['Tomo_Close_proba_1'].describe()

count    138.000000
mean       0.000831
std        0.000554
min        0.000001
25%        0.000572
50%        0.000732
75%        0.000940
max        0.002194
Name: Tomo_Close_proba_1, dtype: float64

In [78]:
print(analytics_df1.shape)
long_swing_df = analytics_df1[ #(analytics_df1['Tomo_Close_proba_1'] > 0.02 ) &
                                (analytics_df1['Tomo_Close_proba_1'] > sorted(analytics_df1['Tomo_Close_proba_1'], reverse=True)[18] ) &
                                ( ( analytics_df1['Adj_Close'] >= analytics_df1['Adj_Close_before_1d'] ) |
                                  ( analytics_df1['Adj_Open'] >= analytics_df1['Adj_Close_before_1d'] ) |
                                  (analytics_df1['Adj_Close'] > analytics_df1['Adj_Close_MA50'] ) |
                                  (analytics_df1['EMA_20'] > analytics_df1['Adj_Close_MA200'] ) 
                                ) &
                                  (analytics_df1['Adj_Vol'] > 700000 ) &
                                  (analytics_df1['Adj_Close']>= 1) &
                                  (analytics_df1['Adj_Close']<= 250) 

                                ].reset_index(drop=True) 

long_swing_df['long_swingable'] = np.where( (long_swing_df['next_8_days_High_max'] >= 1.3 * long_swing_df['Adj_Close']),
                                              True,
                                              False)

greater_than_open_df =  long_swing_df[long_swing_df['long_swingable']==True ].reset_index(drop=True) 

print(long_swing_df.shape) 
print(greater_than_open_df.shape) 
print(f'   win rate: {round(greater_than_open_df.shape[0] / long_swing_df.shape[0],4) }') 
print(f'weekly opps: {round(greater_than_open_df.shape[0] / Weeks_in_analytics_df,2) }') 
 

(138, 35)
(15, 35)
(7, 35)
   win rate: 0.4667
weekly opps: 8.75


In [104]:
special_test_set_res = special_test_set.copy()
special_test_set_res['Tomo_Open'] = special_test_set_res.groupby('Ticker')['Adj_Open'].shift(-1)

special_test_set_res = special_test_set_res[['Ticker','Date', 'Adj_Open', 'Adj_High', 'Tomo_Open', 'Adj_Close', 'Adj_Vol',
                               'price_cat_GT_20', 'price_cat_GT_2_LT_5',
                              'price_cat_GT_5_LT_20', 'price_cat_LT_2', 'ADR_percent_5d', 'ADR_percent_20d',
                               'OHLC_price',  'days_close_surpasses_Shorter', 'next_8_days_High_max', 'Adj_Vol_EMA_prev_5d', 'Adj_Vol_EMA_prev_20d', 'EMA_9_DIV_MA50',
                               'EMA_5_DIV_EMA_20', 'Adj_Close_MA50', 'Adj_Close_MA200', 'EMA_20' ]]

classes = [0,1]
    
special_test_set_res[f'Tomo_Close_proba_0'] = [x[0] for x in special_test_set_pred] 
special_test_set_res[f'Tomo_Close_proba_1'] = [x[1] for x in special_test_set_pred] 

special_test_set_res['Adj_Open_before_1d'] =  special_test_set_res.groupby("Ticker")['Adj_Open'].shift(1) 
special_test_set_res['Adj_High_before_1d'] =  special_test_set_res.groupby("Ticker")['Adj_High'].shift(1) 
special_test_set_res['Adj_Close_before_1d'] =  special_test_set_res.groupby("Ticker")['Adj_Close'].shift(1) 


In [105]:
def get_price_cat(price_cat_LT_2, price_cat_GT_2_LT_5, price_cat_GT_5_LT_20, price_cat_GT_20):
    if price_cat_LT_2:
        return 'LT_2'
    elif price_cat_GT_2_LT_5:
        return 'GT_2_LT_5'
    elif price_cat_GT_5_LT_20:
        return 'GT_5_LT_20'
    else:
        return 'GT_20'

special_test_set_res['price_cat'] = special_test_set_res.apply(lambda row: 
                    get_price_cat( row['price_cat_LT_2'], 
                                   row['price_cat_GT_2_LT_5'], 
                                   row['price_cat_GT_5_LT_20'], 
                                   row['price_cat_GT_20']), axis=1)

#special_test_set_res = special_test_set_res[ (special_test_set_res['ADR_percent_5d'] > 4.5) &
                               #(special_test_set_res['ADR_percent_20d'] > 4.5) &
#                               (special_test_set_res['Tomo_Close_proba_1'] > 0.02 )].reset_index(drop=True)

special_test_set_res = special_test_set_res.sort_values(by=['Date','Tomo_Close_proba_1'],ascending=[True,False]).reset_index(drop=True)
#special_test_set_res = special_test_set_res.groupby(['Date','price_cat']).head(7).reset_index(drop=True)


In [106]:
special_test_set_res['price_cat'].value_counts() 

LT_2          6180
GT_5_LT_20    5443
GT_20         5095
GT_2_LT_5     3382
Name: price_cat, dtype: int64

In [107]:
print(special_test_set_res.shape) 
special_test_set_res.head(2) 

(20100, 29)


,Ticker,Date,Adj_Open,Adj_High,Tomo_Open,Adj_Close,Adj_Vol,price_cat_GT_20,price_cat_GT_2_LT_5,price_cat_GT_5_LT_20,price_cat_LT_2,ADR_percent_5d,ADR_percent_20d,OHLC_price,days_close_surpasses_Shorter,next_8_days_High_max,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_9_DIV_MA50,EMA_5_DIV_EMA_20,Adj_Close_MA50,Adj_Close_MA200,EMA_20,Tomo_Close_proba_0,Tomo_Close_proba_1,Adj_Open_before_1d,Adj_High_before_1d,Adj_Close_before_1d,price_cat
0,BGLC,2023-07-20,3.50,6.23,6.82,4.87,2624800.0,False,True,False,False,64.818001,54.416,4.702,0,9.49,421840.0000,175935.015625,3.192,2.104,0.510,0.74,1.055,0.997806,0.002194,NaN,NaN,NaN,GT_2_LT_5
1,GFAI,2023-07-20,4.18,8.09,7.42,6.44,68650704.0,False,False,True,False,25.271999,10.381,6.103,1,9.90,816739.9375,756605.000000,0.893,1.006,5.409,8.49,4.963,0.997806,0.002194,NaN,NaN,NaN,GT_5_LT_20


In [108]:
special_test_set_res['surpassed_30per'] = special_test_set_res['next_8_days_High_max'] >= 1.3 * special_test_set_res['Adj_Close']
special_test_set_res['surpassed_30per'].value_counts() 

False    19392
True       708
Name: surpassed_30per, dtype: int64

In [109]:
special_test_set_res = special_test_set_res[ ( special_test_set_res['Date']==SPLIT_DATE) &
                                             ( special_test_set_res['ADR_percent_20d']>4) 
                                           ].reset_index(drop=True)
print(special_test_set_res.shape) 


(829, 30)


In [110]:
for price_cat in [ 'price_cat_LT_2', 'price_cat_GT_2_LT_5' , 'price_cat_GT_5_LT_20', 'price_cat_GT_20']:
    l = [x for x in special_test_set_res[special_test_set_res[price_cat]==True].head()['Ticker'].unique()] 
    print(f'{price_cat} : {l} ')


price_cat_LT_2 : ['BBIG', 'BGLC', 'HGEN', 'IRNT', 'MF'] 
price_cat_GT_2_LT_5 : ['BNTC', 'IDAI', 'DRUG', 'TUP', 'ENVB'] 
price_cat_GT_5_LT_20 : ['TNGX', 'MSGM', 'GREE', 'GFAI', 'PRFX'] 
price_cat_GT_20 : ['APLS', 'UPST', 'BBIO', 'AEHR', 'MXL'] 


In [113]:
long_swing_df = special_test_set_res[  (special_test_set_res['ADR_percent_5d'] > 5.5) &
                                       (special_test_set_res['ADR_percent_20d'] > 8) &
                                     (special_test_set_res['Tomo_Close_proba_1'] > \
                                                  sorted(special_test_set_res['Tomo_Close_proba_1'], reverse=True)[80] ) &
                                      # (special_test_set_res['Tomo_Close_proba_1'] > 0.75 ) &
                                     
                                        ( ( special_test_set_res['Adj_Close'] >= special_test_set_res['Adj_Close_before_1d'] ) |
                                          ( special_test_set_res['Adj_Open'] >= special_test_set_res['Adj_Close_before_1d'] ) |
                                          (special_test_set_res['Adj_Close'] > special_test_set_res['Adj_Close_MA50'] ) |
                                          (special_test_set_res['EMA_20'] > special_test_set_res['Adj_Close_MA200'] ) 
                                        ) &
                                      (special_test_set_res['Adj_Vol'] > 600000) &
                                     
                                      ( special_test_set_res['Date'].isin(special_test_set_res['Date'].unique()[-4:])) &
                                      
                                      (special_test_set_res['Adj_Close']>= 1) &
                                      (special_test_set_res['Adj_Close']<= 250) 
    
                            ].reset_index(drop=True) 

long_swing_df['long_swingable'] = np.where( (long_swing_df['next_8_days_High_max'] >= 1.3 * long_swing_df['Adj_Close']),
                                              True,
                                              False)


print(long_swing_df['long_swingable'].value_counts()) 
print("already worked", long_swing_df[long_swing_df['long_swingable']==True]['Ticker'].values)
print(long_swing_df['Ticker'].nunique())
long_swing_df.groupby('price_cat')['Ticker'].unique()


False    18
Name: long_swingable, dtype: int64
already worked []
18


price_cat
GT_2_LT_5                                   [BNTC, IDAI, TUP, RAD, EOSE, BLUE]
GT_5_LT_20                                                              [TNGX]
LT_2          [BGLC, MF, SVFD, YELL, ADMP, COSM, HYZN, FGEN, CRDF, MKFG, BIRD]
Name: Ticker, dtype: object

In [114]:
predicted_EPs = pd.DataFrame() 
predicted_EPs['Ticker'] =  [x for x in long_swing_df['Ticker'].unique()]
print(predicted_EPs.shape)
predicted_EPs.to_pickle(f'{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/predicted_EPs.pkl') 


(18, 1)
